# Access an ASAM ODS EXD-API Plugin

## Prepare Python Environment to Access GRPC Service

In [1]:
# first time you need to install dependencies
if False:
    !python -m pip install --upgrade pip
    !python -m pip install -U grpcio
    !python -m pip install -U grpcio-tools
    #!python -m grpc_tools.protoc --proto_path=. --pyi_out=. --python_out=. ods.proto
    #!python -m grpc_tools.protoc -I. --pyi_out=. --python_out=. --grpc_python_out=. ods_external_data.proto

In [2]:
import os
import pathlib

import grpc
from google.protobuf.json_format import MessageToJson

import ods_pb2
import ods_external_data_pb2
import ods_external_data_pb2_grpc

## EXD-API

The EXD-API plugin is running as a RPC service at a given URL.
Running `exd_api_server.py`´will run the plugin at the given URL.

In [3]:
exd_api_plugin_url = "localhost:50051"

## Import Phase

We will open a MDF4 file using the EXD-API and extract the internal structure of the file to import it into the ASAM ODS server.

In [4]:
data_file_path = os.path.abspath('data/exampleA.raw')
if not os.path.exists(data_file_path):
    raise Exception('Data file is missing')

In [5]:
import_file_url = pathlib.Path(data_file_path).as_uri().replace('///', '//')
import_file_parameters=""
print(import_file_url)

# Will be filled from Structure
access_file_url = None
access_file_parameters = None

file://c:/Users/AKR/github/asam_ods_exd_api_IMCtermite/data/exampleA.raw


### Extract Infos from Structure

The structure contains infos about groups and channels to create corresponding measurements, submatrices and measurement_quantities

In [6]:
with grpc.insecure_channel(exd_api_plugin_url) as channel:
    stub = ods_external_data_pb2_grpc.ExternalDataReaderStub(channel)

    # import file into ASAM ODS Server physical storage
    import_identifier = ods_external_data_pb2.Identifier(
        url=import_file_url,
        parameters=import_file_parameters)
    
    import_handle = stub.Open(import_identifier)
    try:
        structure = stub.GetStructure(
            ods_external_data_pb2.StructureRequest(
                handle=import_handle))
        print(MessageToJson(structure))

        access_file_url = structure.identifier.url
        access_file_parameters = structure.identifier.parameters

        for group in structure.groups:
            group_id = group.id
            for channel in group.channels:
                channel_id = channel.id
    finally:
        stub.Close(import_handle)


{
  "identifier": {
    "url": "file://c:/Users/AKR/github/asam_ods_exd_api_IMCtermite/data/exampleA.raw"
  },
  "name": "exampleA.raw",
  "groups": [
    {
      "name": "Mittelwert",
      "totalNumberOfChannels": "2",
      "numberOfRows": "1",
      "channels": [
        {
          "dataType": "DT_DOUBLE"
        },
        {
          "id": "1",
          "name": "Mittelwert",
          "dataType": "DT_DOUBLE",
          "unitString": "V"
        }
      ],
      "attributes": {
        "variables": {
          "comment": {
            "stringArray": {
              "values": [
                "Me\u00dfreihe 081"
              ]
            }
          },
          "description": {
            "stringArray": {
              "values": [
                "TxBearbeiter - E.Mustermann - 0"
              ]
            }
          },
          "uuid": {
            "stringArray": {
              "values": [
                "253"
              ]
            }
          },
          "orig

## Access Bulk Data

With the stored information the ASAM ODS server can access the bulk data from the EXD-API plugin

In [7]:
with grpc.insecure_channel(exd_api_plugin_url) as channel:
    stub = ods_external_data_pb2_grpc.ExternalDataReaderStub(channel)

    # info from physical storage
    access_group_id = 0
    access_channel_ids = [0, 1]
    access_identifier = ods_external_data_pb2.Identifier(
        url=access_file_url,
        parameters=access_file_parameters)

    # open bulk access
    access_handle = stub.Open(access_identifier)
    try:
        request = ods_external_data_pb2.ValuesRequest(
            handle=access_handle,
            group_id=access_group_id,
            channel_ids=access_channel_ids)

        # read first chunk
        request.start = 0
        request.limit = 3
        values = stub.GetValues(request)
        print(MessageToJson(values))

    finally:
        stub.Close(access_handle)

{
  "channels": [
    {
      "values": {
        "dataType": "DT_DOUBLE",
        "doubleArray": {
          "values": [
            0.0
          ]
        }
      }
    },
    {
      "id": "1",
      "values": {
        "dataType": "DT_DOUBLE",
        "doubleArray": {
          "values": [
            -5.121809677944827e+58
          ]
        }
      }
    }
  ]
}


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INVALID_ARGUMENT
	details = "Channel start index 3 out of range!"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Channel start index 3 out of range!", grpc_status:3, created_time:"2024-05-08T13:24:36.9784042+00:00"}"
>